## Reconhecimento Facial em imagens com openCV

O rosto humano, apesar das variações de pessoa para pessoa, possui uma composição básica que não se altera, lida pelos aplicativos como pontos em comum, que variam de acordo com a complexidade do sistema.

Ao utilizar algoritmos e um software que mapeiam esse padrão nas pessoas, é possível registrar apenas o rosto ou todos os movimentos e empregá-los nas mais diversas funções. Todas possuem o mesmo princípio: detectar um rosto em formas geométricas e logarítmicas e então montá-lo como em um quebra-cabeça. (Kleina, 2021)

Para reconhecer as imagens, usarei a Open Source Computer Vision Library (OpenCV), que é uma biblioteca com mais de 2500 algoritmos de visão computacional e aprendizado de máquina. Possui uma comunidade estimada em mais de 47 mil usuários colaboradores, e o download da biblioteca passa os 18 milhões (OpenCV, 2021).

In [7]:
import cv2

from IPython.display import Image

In [32]:
# para instalar a biblioteca OpenCV use o comando a seguir.
#!pip install opencv-contrib-python
Image(url='/data/img/face-recognition-2.jpg')

## Como são reconhecidos os rostos em imagens?

Para analisar estes arquivos, usarei a técnica do *haar-like cascades features*.
Segundo Antonello (2014) a tradução desta técnica seria algo como “características em cascata do tipo haar”. A palavra *haar* não possui tradução já que o nome deriva dos *wavelets Haar* (ondaleta ou onduleta de Haar) que foram usados no primeiro detector de rosto em tempo real.

*"A principal vantagem da técnica é a baixa necessidade de processamento para realizar a identificação dos objetos, o que se traduz em alta velocidade de detecção. "*\[p.49\]

Foi desenvolvido um método baseado nas onduletas de Haar, que considera as regiões retangulares adjacentes num local específico e então ocorre o processamento da intensidades dos pixels em cada região, calculando a diferença entre eles e categorizando subseções da imagem.

*Por exemplo, digamos que temos imagens com faces humanas. É uma característica comum que entre todas as faces a região dos olhos é mais escura do que a região das bochechas. Portanto, uma característica Haar comum para a detecção de face é um conjunto de dois retângulos adjacentes que ficam na região dos olhos e acima da região das bochechas. A posição desses retângulos é definida em relação a uma janela de detecção que age como uma caixa delimitadora para o objeto alvo (a face, neste caso).*\[p49-50\]

A OpenCV possui modelos pré-treinados com para classificar rostos, olhos, narizes, sorrisos, etc.




### Selecionado a imagem e transformado-a em uma matriz unidimencional (array)

In [34]:
# carrega imagem e transforma em um array numpy
img = cv2.imread('data/img/torcida2.jpg')
# converte a imagem para tons de cinza( e transforma em um array numpy)
img_gray_tone = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [25]:
# carrega padrões de identificação de faces
default_faces_frontal = cv2.CascadeClassifier('data\source\haarcascade_frontalface_default.xml')

In [26]:
# executa a detecção de faces
faces = default_faces_frontal.detectMultiScale(img_gray_tone,
                                              scaleFactor = 1.05,
                                              minNeighbors = 7,
                                              minSize = (20, 20),
                                              flags = cv2.CASCADE_SCALE_IMAGE)

In [27]:
# marca com retângulos as faces reconhecidas
for (x, y, weight, height) in faces:
    cv2.rectangle(img, (x, y), (x + weight, y + height), (0, 255, 255), 7)

In [28]:
# exibe a imagem com uma contagem dos rostos reconhecidos

cv2.imshow(str(len(faces)) + 'rostos encontrados.', img)
cv2.waitKey(0)

-1

In [31]:
# salva imagem marcada

print(f'Foram encontrados {len(faces)} rostos')
cv2.imwrite('results/result_profileface.jpg', img)

# exibe a imagem nesta mesma janela
Image(url = 'results/result_profileface.jpg')

Foram encontrados 160 rostos


## Classidores em cascata

Em vez de aplicar todos os 6000 recursos em uma janela, agrupe os recursos em diferentes estágios de classificadores e aplique um por um. (Normalmente os primeiros estágios conterão um número muito menor de recursos). Se uma janela falhar no primeiro estágio, descarte-o. Não consideramos os recursos restantes nele. Se passar, aplique o segundo estágio de recursos e continue o processo. A janela que passa por todos os estágios é uma região de rosto. 

<p align='center'>
    <img alt="Como a Haar Cascade reconhece rostos (e outros objetos)" style="border-width:0" src="https://www.bogotobogo.com/python/OpenCV_Python/images/FaceDetection/stages.png">
    <br/>
    source: bogotobogo, 2021.
</p>




Quando usamos o classificador em cascata para identificar as imagens no OpenCV, precisamos do método *detectMultiScale()*, cuja sintaxe é a seguinte:

-----------------------

```
cv2.CascadeClassifier.detectMultiScale(image, scaleFactor, minNeighbors, flags, minSize, maxSize)
```
---------------------

1. **imagem:** Matriz do tipo CV_8U contendo uma imagem onde os objetos são detectados.
1. **scaleFactor:** Parâmetro que especifica o quanto o tamanho da imagem é reduzido em cada escala da imagem.
Este fator de escala é usado para criar a pirâmide de escala conforme mostrado na imagem. Suponha que o fator de escala seja 1,03, isso significa que estamos usando um pequeno passo para redimensionar, ou seja, reduzir o tamanho em 3%, aumentamos a chance de um tamanho de correspondência com o modelo para detecção ser encontrado, embora seja caro.
1. **minNeighbors:** parâmetro que especifica quantos vizinhos cada retângulo candidato deve ter para retê-lo. Este parâmetro afetará a qualidade dos rostos detectados: valores mais altos resultam em menos detecções, mas com qualidade superior. Estamos usando 5 no código.
1. **flags:** Parâmetro com o mesmo significado para uma cascata antiga como na função cvHaarDetectObjects. Não é usado para uma nova cascata.
1. **minSize:** Tamanho mínimo do objeto possível. Objetos menores do que isso são ignorados.
1. **maxSize:** Tamanho máximo do objeto possível. Objetos maiores do que isso são ignorados.

--------------

Na nossa imagem podemos observar que ficaram vários rostos sem serem reconhecidos, **vamos melhorar nosso código.**

#### Vamos criar uma função para reconhecer as imagens (e melhorar o reconhecimento)

Até aqui usamos os comandos isolados para melhor compreensão e explicação do tema. Vamos unir os passos anteriores em funções, para agilizar nossa vida.


In [33]:
def imgFaceDetect(arquivo, nome_resultado):
    
    import cv2 as cv

    """
    função que recebe o caminho do arquivo e o nome do arquivo a ser salvo.
    o arquivo recebido é transformado em uma matriz, depois é analisado com
    base nos padrões de reconhecimento de rosto e retorna um arquivo marcado
    e conta a quantidade de rostos que reconheceu.

    Args:
    [user input]
    arquivo ([str]): caminho do arquivo de entrada. (ex.:'dir/img/figura.jpg')
    nome_resultado ([str]): caminho do arquivo de saída. (ex: 'dir/results/figura.jpg') 

    [pre-defition input]
    haar_cascade: arquivo marcado com as onduletas de haar
    faces: percorre a matriz da imagem BGR usando o método de haar. Para isso usamos 
    o v2.CascadeClassifier.detectMultiScale () para encontrar rostos ou olhos, e é definido assim:
        cv2.CascadeClassifier.detectMultiScale (image , scaleFactor , minNeighbors , flags , minSize ,maxSize) 
    pra uma referência completa, consultar:
    https://www.bogotobogo.com/python/OpenCV_Python/python_opencv3_Image_Object_Detection_Face_Detection_Haar_Cascade_Classifiers.php

    """

    # carrega imagem e transforma em um array numpy
    img = cv.imread(arquivo)
    # converte a imagem para tons de cinza (e transforma em um array numpy)
    img_gray_tone = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    # carrega padrões de identificação de faces
    haar_cascade = cv.CascadeClassifier('data\source\haarcascade_frontalface_default.xml')

    # executa a detecção de faces
    # retorna um array com coordenadas, onde cada coordenada representa
    # um rosto detectado
    faces = haar_cascade.detectMultiScale(
        img_gray_tone, 1.01, minNeighbors=7, minSize=(20, 20), flags=cv.CASCADE_SCALE_IMAGE
    )
    # percorre a matriz faces e marca um retângulo seguindo as coordenadas mostradas na matriz
    # a marcação da cor do retângulo está em BGR, neste caso, nossos retângulos serão amarelos
    for (x, y, weight, height) in faces:
        cv.rectangle(img, (x, y), (x + weight, y + height), (0, 255, 255), 2)

    # salva imagem marcada
    cv.imwrite(nome_resultado, img)     
    #mostra quantos rostos foram identificados
    print(f"Foram encontrados {len(faces)} rostos")

In [21]:
imgFaceDetect('data/img/torcida2.jpg', 'results/torcida2.jpg')
Image(url = 'results/torcida2.jpg')

Foram encontrados 265 rostos


In [30]:
imgFaceDetect('data/img/torcida1.jpg', 'results/torcida1.jpg')
Image(url = 'results/torcida1.jpg')

Foram encontrados 146 rostos


In [16]:
# Vou mostrar as demais saídas usando nossa função e chamando o iPython para exibir o resultado
imgFaceDetect('data/img/criancas1.jpg', 'results/criancas1.jpg')
from IPython.display import Image
Image(url = 'results/criancas1.jpg')

Foram encontrados 16 rostos


In [17]:
# Vou mostrar as demais saídas usando nossa função e chamando o iPython para exibir o resultado
imgFaceDetect('data/img/criancas3.jpg', 'results/criancas3.jpg')
Image(url = 'results/criancas3.jpg')

Foram encontrados 9 rostos


In [18]:
# Vou mostrar as demais saídas usando nossa função e chamando o iPython para exibir o resultado
imgFaceDetect('data/img/criancas2.jpg', 'results/criancas2.jpg')
Image(url = 'results/criancas2.jpg')

Foram encontrados 18 rostos


## Considerações finais

É preciso ajustar o fator de scala e outros parâmetros do método **.detectMultiScale()** para cada caso de análise.

Nas imagens com visão ampla, como as da torcida, o resultado da detecção foi consideravelmente bom. Todavia, ao usar os mesmos parâmetros para grupos menores, há uma grande quantidade de falsos positivos, ruídos nas imagens.

O objetivo deste material foi apresentar o básico do reconhecimento facial em imagens, passando de uma visão geral sobre a teoria do reconhecimento facial por visão computacional, até o uso prático da ferramenta openCV.

Caso tenha sugestões ou correções para este material, por favor entre em contato por alguns dos meios citados ao fim deste artigo.

## Referências

ANTONELLO, Ricardo. **Introdução a Visão Computacional com Python e OpenCV**. Engenharia de Controle e Automação do Instituto Federal Catarinense–IFC, campus Luzerna, 2014.

BogoToBogo.[**OBJECT DETECTION : FACE DETECTION USING HAAR CASCADE CLASSFIERS**](https://www.bogotobogo.com/python/OpenCV_Python/python_opencv3_Image_Object_Detection_Face_Detection_Haar_Cascade_Classifiers.php). Acessado em 06/04/2021.

Kleinam, Nilton - [**Entenda como funcionam os sistemas de reconhecimento facial**](https://www.terra.com.br/noticias/tecnologia/hardware-e-software/entenda-como-funcionam-os-sistemas-de-reconhecimento-facial,da888a19d13ea310VgnCLD200000bbcceb0aRCRD.html). Acessado em 30/03/2021

OpenCV.[**About**](https://opencv.org/about/). Acessado em 01/04/2021.

---------------
**crédito das imagens** 

- Imagem de <a href="https://br.freepik.com/seventyfour">seventyfour</a>
in <a href="https://br.freepik.com/fotos-premium/criancas-americanas-inter-raciais_8773881.htm">FreePick</a>

- Imagem de <a href="https://pixabay.com/pt/users/383961-383961/?utm_source=link-attribution&amp;utm_medium=referral&amp;utm_campaign=image&amp;utm_content=428909">bigter choi</a> in <a href="https://pixabay.com/pt/photos/crian%C3%A7as-menina-menino-homem-%C3%ADndia-428909/">Pixabay</a>

- Imagem de <a href="https://pixabay.com/pt/users/keithjj-2328014/?utm_source=link-attribution&amp;utm_medium=referral&amp;utm_campaign=image&amp;utm_content=1488213">Keith Johnston</a>(
 <a href="https://pixabay.com/pt/photos/multid%C3%A3o-f%C3%A3s-de-esportes-1584115/">aqui</a> e <a href="https://pixabay.com/pt/photos/multid%C3%A3o-de-pessoas-multid%C3%A3o-1488213/"> aqui.)</a>

- Imagem de <a href="https://www.johnsnowlabs.com/the-math-behind-face-recognition/">Mohamed Tharwat</a> THE MATH BEHIND FACE RECOGNITION

- Imagem de <a href="https://pixabay.com/pt/users/ean254-4188723/?utm_source=link-attribution&utm_medium=referral&utm_campaign=image&utm_content=3137670">Ian Ingalula</a> in [pixabay](https://pixabay.com/pt/photos/pessoas-grupo-crian%C3%A7as-menino-3137670/)



## Contato

**Author:** Jonas Araujo de Avila Campos

**Confira mais projetos: [AQUI](https://jonasaacampos.github.io/portifolio/)**

<p align='center'>

  <a href='https://github.com/jonasaacampos'>
    <img src='https://img.shields.io/badge/GitHub-100000?style=for-the-badge&logo=github&logoColor=white'/>
  </a>

  <a href='https://www.linkedin.com/in/jonasaacampos/'>
    <img src='https://img.shields.io/badge/LinkedIn-0077B5?style=for-the-badge&logo=linkedin&logoColor=white'/>
  </a>

  <a href='https://www.facebook.com/jonasaacampos'>
    <img src='https://img.shields.io/badge/Facebook-1877F2?style=for-the-badge&logo=facebook&logoColor=white'/>
  </a>
  
</p>